## $\Rightarrow$ Notebook to test the code used to generate the covariance matrix using GWFish and including KAGRA (K1) into the network.

In [1]:
#-- import modules -

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
from itertools import combinations
import os
import sys

from pycbc.detector import add_detector_on_earth
from pycbc.waveform import get_waveform_filter_length_in_time as duration
from pycbc.psd.analytical import aLIGOAPlusDesignSensitivityT1800042, AdVDesignSensitivityP1200087, KAGRADesignSensitivityT1600593, KAGRA, AdvVirgo
from pycbc.psd.analytical import aLIGODesignSensitivityP1200087, aLIGOZeroDetHighPower
from pycbc.cosmology import cosmological_quantity_from_redshift
from pycbc.cosmology import redshift
from pycbc.cosmology import distance_from_comoving_volume, redshift_from_comoving_volume
from pycbc.waveform.generator import FDomainCBCGenerator, FDomainDetFrameGenerator
from pycbc.types.frequencyseries import load_frequencyseries
from pycbc.filter import sigma
from pycbc.psd.estimate import interpolate
from pycbc.types import FrequencySeries
from pycbc.noise.gaussian import frequency_noise_from_psd
from pycbc.conversions import mchirp_from_mass1_mass2, eta_from_mass1_mass2

import gwfast.gwfastGlobals as glob
import copy
from gwfast.waveforms import IMRPhenomD_NRTidalv2, TaylorF2_RestrictedPN
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams

### $\rightarrow$ Original Function

In [2]:
#-- subroutine to calculate covariance matrix ---
def generate_covmat(event, fiducial_params, ifos):
    
    """Function to generate covariance matrix using gwfast package developed by Iacovelli et al.
    For more info about gwfast, please visit https://github.com/CosmoStatGW/gwfast
    
    Parameters
    ----------
    fiducial_params: dictionary of parameters at which to calculate the metric (to be given in the following order)
    ['Mc', 'eta', 'chi1z', 'chi2z', 'iota', 'phi', 'theta', 'dL', 'psi', 'tGPS']
    ifos: list of interferometers
    
    Returns
    -------
    cov_mat: a covarince matrix in intrinsic parameters (['Mc', 'eta', 'chi1z', 'chi2z'])"""

    alldetectors = copy.deepcopy(glob.detectors)
    
    # select only LIGO and Virgo
    
    LVdetectors = {}
    
    for ifo in ifos:
        if(ifo=='V1'):
            ifo = 'Virgo'
        LVdetectors[ifo] = alldetectors[ifo]
    
    for ifo in ifos:
        if(ifo=='V1'):
            ifo_mod = 'Virgo'
            LVdetectors[ifo_mod]['psd_path'] = os.getcwd() + '/psd/{}_PSD_{}.txt'.format(event, ifo)
        else:
            LVdetectors[ifo]['psd_path'] = os.getcwd() + '/psd/{}_PSD_{}.txt'.format(event, ifo)

    myLVSignals = {}

    for d in LVdetectors.keys():

        myLVSignals[d] = GWSignal(TaylorF2_RestrictedPN(), 
                    psd_path=LVdetectors[d]['psd_path'],
                    detector_shape = LVdetectors[d]['shape'],
                    det_lat= LVdetectors[d]['lat'],
                    det_long=LVdetectors[d]['long'],
                    det_xax=LVdetectors[d]['xax'], 
                    verbose=False,
                    useEarthMotion = False,
                    fmin=10.,
                    IntTablePath=None, is_ASD=False) 

    myLVNet = DetNet(myLVSignals)
    
    GW170817_dict = {}
    
    for key, val in fiducial_params.items():
        if(key=='tGPS'):
            GW170817_dict[key] = np.array([val])
        elif(key=='dL'):
            GW170817_dict[key] = np.array([val*1e-3])
        elif(key=='theta'):
            GW170817_dict[key] = np.array([np.pi/2 - val])
        else:
            GW170817_dict[key] = np.array([val])
        
    GW170817_dict.update(dict(Phicoal=np.array([0.])))
    GW170817_dict.update(dict(Lambda1=np.array([0.])))
    GW170817_dict.update(dict(Lambda2=np.array([0.])))
    
    SNR = myLVNet.SNR(GW170817_dict)
    print('SNR for GW170817 is %.2f'%SNR)
    
    totF = myLVNet.FisherMatr(GW170817_dict, df= 1/360, use_m1m2=True) # replace df=1/360 by an appropriate variable name
    ParNums = TaylorF2_RestrictedPN().ParNums
    newFish, newPars = fixParams(totF, ParNums, ['iota', 'phi', 'theta', 'dL', \
                                                                     'tcoal', 'Phicoal', 'psi'])
    newCov = CovMatr(newFish)[0]
    cov_mat = np.empty((4,4))
    
    for i in range(4):
        
        for j in range(4):
            
            cov_mat[i,j] = newCov.reshape(4,4)[i,j]
                                        
    return cov_mat

### $\rightarrow$ Writing a new function

In [3]:
ifos = ['L1', 'H1', 'V1', 'K1']

In [4]:
#-- TO STORE THE LOCATION COORDINATES OF GW DETECTORS --

alldetectors = copy.deepcopy(glob.detectors)

# select LIGO, Virgo and KAGRA detectors

LVK_detectors = {}

for ifo in ifos:
    
    if(ifo=='V1'):
        
        ifo = 'Virgo'
        
        LVK_detectors[ifo] = alldetectors[ifo]
    
    elif(ifo=='K1'):
        
        ifo = 'KAGRA'
        
        LVK_detectors[ifo] = alldetectors[ifo]
        
    else:
        
        LVK_detectors[ifo] = alldetectors[ifo]
        

In [5]:
LVK_detectors['Virgo']

{'lat': 43.631, 'long': 10.504, 'xax': 115.56756342034298, 'shape': 'L'}

In [6]:
for ifo in ifos:
    
    if(ifo=='V1'):
        
        ifo_mod = 'Virgo'
        LVK_detectors[ifo_mod]['psd_path'] = os.getcwd() + '/../../../../psds/PSD_{}_10Hz_to_2048Hz_nonzero.txt'.format(ifo_mod)
        
    elif(ifo=='K1'):
        
        ifo_mod = 'KAGRA'
        LVK_detectors[ifo_mod]['psd_path'] = os.getcwd() + '/../../../../psds/PSD_{}_10Hz_to_2048Hz_nonzero.txt'.format(ifo_mod)
        
    else:
        
        LVK_detectors[ifo]['psd_path'] = os.getcwd() + '/../../../../psds/PSD_O5_10Hz_to_2048Hz_nonzero.txt'
        

In [7]:
LVK_detectors['KAGRA']

{'lat': 36.412,
 'long': 137.306,
 'xax': 15.396,
 'shape': 'L',
 'psd_path': '/home/sachins/work/PROJECTS/GW/bns_localization_with_LIGO_Aundha/localize_sim_bns/PE_Network_A0_O4/PE_combinations/PE_LHVK/../../../../psds/PSD_KAGRA_10Hz_to_2048Hz_nonzero.txt'}

In [7]:
#-- GENERATE CBC SIGNALS AS SEEN IN EACH DETECTOR AND NETWORK OF DETECTOR (using DetNet object) --

myLVK_Signals = {}

for d in LVK_detectors.keys():

    myLVK_Signals[d] = GWSignal(TaylorF2_RestrictedPN(), 
                                psd_path=LVK_detectors[d]['psd_path'],
                                detector_shape = LVK_detectors[d]['shape'],
                                det_lat= LVK_detectors[d]['lat'],
                                det_long=LVK_detectors[d]['long'],
                                det_xax=LVK_detectors[d]['xax'], 
                                verbose=False,
                                useEarthMotion = False,
                                fmin=10.,
                                IntTablePath=None, is_ASD=False) 

myLVK_Net = DetNet(myLVK_Signals)


In [8]:
#-- Injection Parameters --

m1_src = 1.387
m2_src = 1.326
s1z = 0.0001294116725339975
s2z = 3.5461352589076355e-05

iota, pol = 0, 0
tc = 1597236082 

ra = 3.448296944257913 
dec = 0.44491231446252155 
dL = 168.65546273003267 
z = 0.03706738893253226

In [9]:
#-- Detector Frame masses --

m1 = m1_src * (1+z)
m2 = m2_src * (1+z)

#-- Derived Params --

mchirp = mchirp_from_mass1_mass2(m1, m2)
eta = eta_from_mass1_mass2(m1, m2)

In [10]:
#-- DEFINING FIDUCIAL PARAMS --

fiducial_params = {}

fiducial_params['Mc'] = mchirp
fiducial_params['eta'] = eta
fiducial_params['chi1z'] = s1z
fiducial_params['chi2z'] = s2z
fiducial_params['iota'] = iota
fiducial_params['phi'] = ra
fiducial_params['theta'] = dec
fiducial_params['dL'] = dL
fiducial_params.update(dict(psi=pol, tGPS=tc)) 

In [11]:
myEvent_dict = {}
    
for key, val in fiducial_params.items():
    
    if(key=='tGPS'):
        myEvent_dict[key] = np.array([val])
    elif(key=='dL'):
        myEvent_dict[key] = np.array([val*1e-3])
    elif(key=='theta'):
        myEvent_dict[key] = np.array([np.pi/2 - val])
    else:
        myEvent_dict[key] = np.array([val])

myEvent_dict.update(dict(Phicoal=np.array([0.])))
myEvent_dict.update(dict(Lambda1=np.array([0.])))
myEvent_dict.update(dict(Lambda2=np.array([0.])))


In [13]:
SNR = myLVK_Net.SNR(myEvent_dict)
print('SNR for myEvent is %.2f'%SNR)

totF = myLVK_Net.FisherMatr(myEvent_dict, df= 1/1500, use_m1m2=True) # replace df=1/360 by an appropriate variable name

ParNums = TaylorF2_RestrictedPN().ParNums

newFish, newPars = fixParams(totF, ParNums, ['iota', 'phi', 'theta', 'dL', \
                                                                 'tcoal', 'Phicoal', 'psi'])

newCov = CovMatr(newFish)[0]

cov_mat = np.empty((4,4))

for i in range(4):

    for j in range(4):

        cov_mat[i,j] = newCov.reshape(4,4)[i,j]



Adding tcoal from tGPS


ValueError: interpolating from IERS_Auto using predictive values that are more
than 30.0 days old.

Normally you should not see this error because this class
automatically downloads the latest IERS-A table.  Perhaps you are
offline?  If you understand what you are doing then this error can be
suppressed by setting the auto_max_age configuration variable to
``None``:

  from astropy.utils.iers import conf
  conf.auto_max_age = None


In [ ]:
cov_mat

In [59]:
newCov.reshape(4,4)

array([[ 0.18771961, -0.17786184,  0.60966047, -0.63817667],
       [-0.17786184,  0.16852173, -0.57764847,  0.60466739],
       [ 0.60966047, -0.57764847,  2.73407388, -2.8700274 ],
       [-0.63817667,  0.60466739, -2.8700274 ,  3.0128041 ]],
      dtype=float128)